# YOLOv9 - O Estado da Arte para Detecção de Objetos

A detecção de objetos é uma tarefa fundamental em visão computacional, e tem visto um progresso extraordinário devido ao avanço dos modelos de *deep learning*.

Nesse escopo de problemas, a família YOLO (*You Only Look Once*) se destaca na vanguarda, reconhecida por sua excepcional velocidade de inferência em tempo real e precisão.

Se você ainda estava absorvendo e testando as aplicações da YOLOv8, eu tenho uma novidade para você. De maneira quase que inesperada, fomos presenteados com a novidade de uma YOLOv9 - possivelmente assumindo a posição de modelo no estado da arte.

<center>
<img src="https://sigmoidal.ai/wp-content/uploads/2024/02/yolov9-banner.png" width=600>
</center>


## *YOLOv9: Learning What You Want to Learn Using Programmable Gradient Information - Wang, Chien-Yao  and Liao, Hong-Yuan Mark (2024)*

Abordagens existentes de deep learning em detecção de objetos frequentemente enfatizam o design de arquiteturas de rede complexas ou a elaboração de funções objetivo especializadas. No entanto, elas tendem a negligenciar um problema crucial: os dados sofrem significativa perda de informação durante sua jornada pelas camadas da rede.

Este "gargalo de informação" corrompe os fluxos de gradiente durante a retropropagação, potencialmente levando a atualizações tendenciosas e predições imprecisas do modelo.

<center>
<img src="https://github.com/carlosfab/yolov9/raw/main/figure/performance.png" width=500></center>

Para enfrentar este desafio, os autores do paper propuseram o conceito de Informação de Gradiente Programável (PGI). PGI introduz um ramo reversível auxiliar que gera gradientes confiáveis, preservando características cruciais de entrada essenciais para a tarefa alvo. Ele evita a diluição semântica que pode ocorrer em técnicas tradicionais de supervisão profunda. Ao efetivamente "programar" o fluxo de informação de gradiente através da rede, PGI facilita resultados de treinamento ideais.

Além disso, os autores também introduziram a Rede de Agregação de Camadas Eficiente Generalizada (GELAN). GELAN equilibra meticulosamente a contagem de parâmetros, complexidade computacional, precisão e velocidade de inferência, oferecendo flexibilidade para implantação em diversos dispositivos. Em um feito notável, GELAN alcança utilização de parâmetros superior em comparação até mesmo com modelos de ponta que dependem de convoluções depth-wise.

**A sinergia de PGI e GELAN forma a espinha dorsal do YOLOv9.** Experimentos exaustivos no dataset MS COCO demonstram o domínio do YOLOv9. Especificamente:

* Desempenho Inigualável: YOLOv9 supera todos os detectores de objetos em tempo real existentes em métricas-chave.
* Eficiência: Supera até mesmo modelos pré-treinados em grandes conjuntos de dados, enquanto é treinado do zero.
* Leveza: PGI capacita modelos leves, tornando a detecção de objetos acessível para aplicações cotidianas.


Esta aula é apenas uma introdução sobre o modelo YOLOv9. A verdade é que toda a comunidade ainda está estudando o artigo, e validando as informações que têm chegado em um ritmo acelerado.

Se você quer se manter à frente no mercado de Visão Computacional, acompanhe esta primeira introdução que eu preparei para você!

In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


Para clonar o repositório do YOLOv9 e instalar as dependências necessárias, siga os passos abaixo:


In [ ]:
# Clona o repositório do YOLOv9
!git clone https://github.com/carlosfab/yolov9.git

# Muda o diretório de trabalho atual para o repositório YOLOv9 clonado
%cd yolov9

# Instala as dependências necessárias do YOLOv9 a partir do arquivo requirements.txt
!pip install -r requirements.txt -q

Cloning into 'yolov9'...
remote: Enumerating objects: 168, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 168 (delta 27), reused 22 (delta 18), pack-reused 127
Receiving objects: 100% (168/168), 620.20 KiB | 9.40 MiB/s, done.
Resolving deltas: 100% (67/67), done.
/content/yolov9/yolov9
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.1 MB/s eta 0:00:00


In [ ]:
# Importa bibliotecas necessárias
import sys
import requests
from tqdm.notebook import tqdm
from pathlib import Path
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams


# Configuração de diretórios para código e dados
CODE_FOLDER = Path("..").resolve()  # Diretório do código
WEIGHTS_FOLDER = CODE_FOLDER / "weights"  # Diretório para pesos do modelo
DATA_FOLDER = CODE_FOLDER / "data"  # Diretório para dados

# Cria os diretórios para pesos e dados, se não existirem
WEIGHTS_FOLDER.mkdir(exist_ok=True, parents=True)
DATA_FOLDER.mkdir(exist_ok=True, parents=True)

# Adiciona o diretório do código ao path do Python para importação de módulos
sys.path.append(str(CODE_FOLDER))

rcParams['figure.figsize'] = 15, 15
%matplotlib inline

In [ ]:
# URLs dos arquivos de pesos
weight_files = [
    "https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-c.pt",
    "https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-e.pt",
    "https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-c.pt",
    "https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-e.pt"
]

# Itera sobre a lista de URLs para baixar os arquivos de pesos
for i, url in enumerate(weight_files, start=1):
    filename = url.split('/')[-1]
    response = requests.get(url, stream=True)
    total_size_in_bytes = int(response.headers.get('content-length', 0))
    block_size = 1024  # 1 Kilobyte
    progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True, desc=f"Baixando arquivo {i}/{len(weight_files)}: {filename}")
    with open(WEIGHTS_FOLDER / filename, 'wb') as file:
        for data in response.iter_content(block_size):
            progress_bar.update(len(data))
            file.write(data)
    progress_bar.close()

Baixando arquivo 1/4: yolov9-c.pt:   0%|          | 0.00/103M [00:00<?, ?iB/s]

Baixando arquivo 2/4: yolov9-e.pt:   0%|          | 0.00/140M [00:00<?, ?iB/s]

Baixando arquivo 3/4: gelan-c.pt:   0%|          | 0.00/51.5M [00:00<?, ?iB/s]

Baixando arquivo 4/4: gelan-e.pt:   0%|          | 0.00/117M [00:00<?, ?iB/s]

In [ ]:
# URL da imagem de teste
url = 'https://sigmoidal.ai/wp-content/uploads/2022/11/314928609_1293071608150779_8666358890956473002_n.jpg'

# Faz a requisição para obter a imagem
response = requests.get(url)

# Define o caminho do arquivo onde a imagem será salva dentro do DATA_FOLDER
image_path = DATA_FOLDER / "test_image.jpg"

# Salva a imagem no diretório especificado
with open(image_path, 'wb') as f:
    f.write(response.content)

In [130]:
!python detect.py --weights {WEIGHTS_FOLDER}/yolov9-e.pt --conf 0.1 --source {DATA_FOLDER}/test_image.jpg --device cpu

detect: weights=['/content/weights/yolov9-e.pt'], source=/content/data/test_image.jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=cpu, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLO 🚀 6b38221 Python-3.10.12 torch-2.5.1+cpu CPU

/content/yolov9/models/experimental.py:243: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped t

In [136]:
import torch
from pathlib import Path
from pprint import pprint
from models.common import DetectMultiBackend
from utils.general import non_max_suppression, scale_boxes

def letterbox(im, new_shape=(640, 640), color=(114, 114, 114), auto=True, scaleFill=False, scaleup=True, stride=32):
    # Resize and pad image while meeting stride-multiple constraints
    shape = im.shape[:2]  # current shape [height, width]
    if isinstance(new_shape, int):
        new_shape = (new_shape, new_shape)

    # Scale ratio (new / old)
    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
    if not scaleup:  # only scale down, do not scale up (for better val mAP)
        r = min(r, 1.0)

    # Compute padding
    ratio = r, r  # width, height ratios
    new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]  # wh padding
    if auto:  # minimum rectangle
        dw, dh = np.mod(dw, stride), np.mod(dh, stride)  # wh padding
    elif scaleFill:  # stretch
        dw, dh = 0.0, 0.0
        new_unpad = (new_shape[1], new_shape[0])
        ratio = new_shape[1] / shape[1], new_shape[0] / shape[0]  # width, height ratios

    dw /= 2  # divide padding into 2 sides
    dh /= 2

    if shape[::-1] != new_unpad:  # resize
        im = cv2.resize(im, new_unpad, interpolation=cv2.INTER_LINEAR)
    top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
    left, right = int(round(dw - 0.1)), int(round(dw + 0.1))
    im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)  # add border
    return im, ratio, (dw, dh)

def detect(weights, img_np, conf_thres=0.25, iou_thres=0.45, imgsz=(640, 640), im0=None, device='', classes=None, agnostic_nms=False):
    """
    Realiza detecção de objetos em uma imagem usando YOLO.

    Args:
        weights (str): Caminho para os pesos do modelo.
        img_np (numpy.ndarray): Imagem em formato NumPy (H, W, C).
        conf_thres (float): Limite de confiança para detecção.
        iou_thres (float): Limite IOU para supressão não máxima.
        imgsz (tuple): Tamanho da imagem para inferência (altura, largura).
        device (str): Dispositivo para executar o modelo ('cpu' ou 'cuda').
        classes (list): Lista de classes para filtrar (opcional).
        agnostic_nms (bool): Ativar supressão não máxima independente de classe.

    Returns:
        list: Lista de detecções no formato [x1, y1, x2, y2, conf, cls].
    """
    # Inicializa o modelo
    device = torch.device(device if torch.cuda.is_available() else 'cpu')
    model = DetectMultiBackend(weights, fp16=False)
    stride,names, pt = model.stride, model.names, model.pt
    # Preprocessa a imagem
    img = torch.from_numpy(img_np).to(model.device)
    print('dasdas', type(im), im.shape[2:])
    img = img.half() if model.fp16 else img.float()  # uint8 to fp16/32
    img /= 255  # 0 - 255 to 0.0 - 1.0
    if len(img.shape) == 3:
        img = img[None]  # expand for batch dim

    # Realiza inferência
    pred = model(img)

    pred = pred[0][1] if isinstance(pred[0], list) else pred[0]
    # Aplica NMS (Non-Maximum Suppression)
    pred = non_max_suppression(pred, conf_thres, iou_thres, classes=classes, agnostic=agnostic_nms)

    # Escala as caixas de volta para o tamanho original da imagem
    det = pred[0]
    if det is not None and len(det):
        print('informação',im.shape[2:], im0.shape)
        det[:, :4] = scale_boxes(img.shape[2:], det[:, :4], im0.shape).round()

    return det.tolist() if det is not None else []


In [137]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# Carregar a imagem como NumPy
image = cv2.imread("/content/data/test_image.jpg")
im = letterbox(image,[640, 640],(114, 114, 114), True)[0]
im = im.transpose((2, 0, 1))[::-1]
im = np.ascontiguousarray(im)
weights_path = "/content/weights/yolov9-e.pt"
detections = detect(weights=weights_path, img_np=im, im0=image, conf_thres=0.5, iou_thres=0.4)
for det in detections:
    print(f"Caixa: {det[:4]}, Confiança: {det[4]}, Classe: {det[5]}")

/content/yolov9/models/experimental.py:243: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(attempt_download(w), map_location='cpu')  # load
Fusing layers...

dasdas <class 'numpy.ndarray'> (608,)
informação (608,) (1535, 1440, 3)
Caixa: [1111.0, 639.0, 1438.0, 962.0], Confiança: 0.9316539764404297, Classe: 0.0
Caixa: [832.0, 658.0, 1195.0, 948.0], Confiança: 0.9074289202690125, Classe: 0.0
Caixa: [301.0, 648.0, 567.0, 933.0], Confiança: 0.9073002934455872, Classe: 0.0
Caixa: [589.0, 644.0, 840.0, 924.0], Confiança: 0.9039127826690674, Classe: 0.0
Caixa: [349.0, 848.0, 388.0, 970.0], Confiança: 0.8370159268379211, Classe: 39.0
Caixa: [1002.0, 847.0, 1038.0, 962.0], Confiança: 0.8335368633270264, Classe: 39.0
Caixa: [237.0, 861.0, 286.0, 1000.0], Confiança: 0.8304003477096558, Classe: 39.0
Caixa: [194.0, 862.0, 238.0, 1000.0], Confiança: 0.8256590962409973, Classe: 39.0
Caixa: [293.0, 852.0, 345.0, 973.0], Confiança: 0.7557481527328491, Classe: 39.0
Caixa: [1194.0, 851.0, 1231.0, 974.0], Confiança: 0.7499076724052429, Classe: 39.0
Caixa: [884.0, 844.0, 917.0, 918.0], Confiança: 0.7104252576828003, Classe: 39.0
Caixa: [587.0, 842.0, 621.0, 918